In [1]:
import torch
from torch import nn
import numpy as np
import os
import numpy as np
import itertools
import math, random
random.seed = 42
import numpy as np
import open3d as o3d

import numpy as np
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

import pygem
print(pygem.__version__)
from visualize import * 
from plyfile import PlyData
import scipy.spatial.distance
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from pygem import FFD
from visualize import pcshow,pc_show_multi,visualize_rotate,pcwrite
from point_utils import *
from bernsetin import *
from FFD import _calculate_ffd
from path import Path
import torch
from torch import nn
from model.model import *
import re
# from emd__ import emd_module
import argparse

2.0.0
2.0.0


In [2]:
#循环数据集
#写一个txt
#get 文件名
#变形,两种方式
#


def np_to_tensor(x):
    return torch.from_numpy(x.astype(np.float32))

def deform_point(tensor,classifier,deform_net1,deform_net2):
    classifier.eval()
    deform_net1.eval()
    deform_net2.eval()


    feats,_,_ = classifier(tensor)   
    norm_feat = torch.nn.functional.normalize(feats[0], p=2.0, dim = -1)
    norm_feat = norm_feat.unsqueeze(0)

    dp1 = deform_net1(norm_feat)
    dp1 = dp1.detach().numpy()


    dp2 = deform_net2(norm_feat)
    dp2 = dp2.detach().numpy()

    return dp1[0],dp2[0]

In [3]:
folder_name = 'Deformed_ModelNet40'

In [4]:

path = Path('/Users/wuhongyu/code/Dataset/ModelNet40')
data_set_path = '/Users/wuhongyu/code/Dataset/ModelNet40'
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
model_path = 'model/pointnet_0.0001_2000_0.8_FFD_Contrast_mixup_125_train-64_3layer_128_chamfer'
deform_net_1_path =  model_path+ '/deform/deform_net_1.pth.tar'
deform_net_2_path =  model_path+ '/deform/deform_net_2.pth.tar'

# deform_net_1_path = 'deform_net_1.pth.tar'
# deform_net_2_path =  'deform_net_2.pth.tar'

back_bone =  model_path+'/best/best_model.pth.tar'

In [5]:
def find_max_layer(state_dict):
    numbers_after_fc = [int(re.search('fc(\d+)', item).group(1)) for item in state_dict if 'fc' in item]
    return max(numbers_after_fc)
deform_net_sd = torch.load(deform_net_1_path,map_location=torch.device('cpu'))['state_dict']
max_layer = find_max_layer(deform_net_sd)

number_cp = int(model_path.split('_')[7])
number_cp_per_axis = round(number_cp ** (1/3))

In [9]:
deform_net_map = {
        1: Deform_Net_1layer,
        2: Deform_Net_2layer,
        3: Deform_Net_3layer
    }

deform_net1 =  deform_net_map[max_layer](in_features=128,out_features=(number_cp_per_axis)**3 * 3)
deform_net1.load_state_dict(torch.load(deform_net_1_path,map_location=torch.device('cpu'))['state_dict'],strict=False)

deform_net2 =  deform_net_map[max_layer](in_features=128,out_features=(number_cp_per_axis)**3 * 3)
deform_net2.load_state_dict(torch.load(deform_net_2_path,map_location=torch.device('cpu'))['state_dict'],strict=False)

classifier = Contrastive_PointNet()
classifier.load_state_dict(torch.load(back_bone,map_location=torch.device('cpu'))['state_dict'],strict=True)

<All keys matched successfully>

In [10]:
limit_count = 0

In [11]:
folder_name = 'Deformed_ModelNet40'
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.mkdir(folder_name)
    print(f"Folder {folder_name} created successfully.")

In [50]:
limit_count = 0
for cls in folders:
        print(cls)
        if '.' in cls or limit_count>30:
            limit_count = 0
            continue
        # get current class folder
        cls_folder_name = cls
        cls_folder_path = folder_name+'/'+cls_folder_name

        if not os.path.exists(cls_folder_path):
        # If it doesn't exist, create the folder
            os.mkdir(cls_folder_path)

        if not os.path.exists(cls_folder_path+'/'+'train'):
            os.mkdir(cls_folder_path+'/'+'train')

        if not os.path.exists(cls_folder_path+'/'+'test'):
            os.mkdir(cls_folder_path+'/'+'test')
            # print(f"Folder {cls_folder_name} created successfully.")
        # get all the files in the folder
        file_list = os.listdir(path/cls/'train')

        for file in file_list:
            with open(os.path.join(data_set_path+'/'+cls+'/train',file),'r') as f:
                    # with open(path/cls/'train'/file, 'rb') as f:
                        data  = read_off(f)
                        verts, faces  = data 
                        b,p,xyz= _calculate_ffd(np.array(verts),faces,n=5,n_samples=3072)
            # get origin 3d img
            origin = np.matmul(b,p)
            origin_tensor = np_to_tensor(np.array(origin)).unsqueeze(0)
            origin_tensor = origin_tensor.transpose(2, 1)

            # get deformed control points
            # dp1,dp2 = deform_point(origin_tensor,classifier,deform_net1,deform_net2)
            dp_delta1 = np.random.rand(p.shape[0],p.shape[1])
            dp_delta2 = np.random.rand(p.shape[0],p.shape[1])

                    # get the new 3d img
            new1 =  np.matmul(b,p+dp_delta1)
            new1 = Normalize()(new1)
            new2 =  np.matmul(b,p+dp_delta2)
            new2 = Normalize()(new2)
            mixup = 0.5 * new2 + 0.5+ new1
            mixup = Normalize()(mixup)
            file_name = file.split('.')[0]
            save_path = os.path.join(folder_name+'/'+cls+'/train',file_name)
            np.save(file=save_path+'.npy',arr=xyz)
            np.save(file=save_path+'_ffd1.npy',arr=new1)
            np.save(file=save_path+'_ffd2.npy',arr=new2)
            np.save(file=save_path+'_mixup.npy',arr=mixup)
            print(save_path)
            limit_count +=1






airplane
*****************
Deformed_ModelNet40/airplane/train/airplane_0487
*****************
Deformed_ModelNet40/airplane/train/airplane_0493
*****************
Deformed_ModelNet40/airplane/train/airplane_0322
*****************
Deformed_ModelNet40/airplane/train/airplane_0444
*****************
Deformed_ModelNet40/airplane/train/airplane_0450
*****************
Deformed_ModelNet40/airplane/train/airplane_0336
*****************
Deformed_ModelNet40/airplane/train/airplane_0478
*****************
Deformed_ModelNet40/airplane/train/airplane_0120
*****************
Deformed_ModelNet40/airplane/train/airplane_0134
*****************
Deformed_ModelNet40/airplane/train/airplane_0108
*****************
Deformed_ModelNet40/airplane/train/airplane_0097


KeyboardInterrupt: 

In [46]:
file_name = file.split('.')[0]
os.path.join(folder_name+'/'+cls+'/train',file_name)+'_ffd1.npy'

'Deformed_ModelNet40/xbox/train/xbox_0044_ffd1.npy'

In [48]:
save_path

'Deformed_ModelNet40/xbox/train/xbox_0014'

In [74]:
ffd_1=np.load('Deformed_ModelNet40/airplane/train/airplane_0444_ffd1.npy')
ffd_2=np.load('Deformed_ModelNet40/airplane/train/airplane_0444_ffd2.npy')
ffd_3=np.load('Deformed_ModelNet40/airplane/train/airplane_0444_mixup.npy')

In [75]:
pc_show_multi(ffd_1.T,ffd_3.T)

In [55]:
len(xyz)

3072

In [ ]:
# 引入control points 的选项